In [1]:
!pip install --upgrade duckdb
!pip install duckdb-engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 25.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.1.0
    Uninstalling duckdb-1.1.0:
      Successfully uninstalled duckdb-1.1.0

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import duckdb
import os

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

Matplotlib is building the font cache; this may take a moment.


In [3]:
dataset_path = 'buckets/b1/datasets/'
dataset_file = 'competencia_03_crudo.csv.gz'

In [5]:
%%sql
create or replace table competencia_03_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}", COMPRESSION='GZIP')

,Success


In [6]:
%%sql
create or replace table competencia_03 as
with periodos as (
    select distinct foto_mes from competencia_03_crudo
), clientes as (
    select distinct numero_de_cliente from competencia_03_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , if(mes_2 = 1,'CONTINUA', if(mes_1 = 0,'BAJA+1',if(mes_2 = 0,'BAJA+2',Null))) as clase_ternaria
    from todo t
    left join competencia_02_crudo c using (numero_de_cliente, foto_mes)
) select
  * EXCLUDE (mes_0, mes_1, mes_2)
from clase_ternaria
where mes_0 = 1

,Success


In [7]:
%sql select count(*) from competencia_03

,count_star()
0,4901237


In [8]:
%sql select count(*) from competencia_03_crudo

,count_star()
0,4901237


In [9]:
%%sql
PIVOT competencia_03
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,201901,688,718,123346
1,201902,720,688,124391
2,201903,688,760,124988
3,201904,759,579,125658
4,201905,580,660,126419
5,201906,662,608,127916
6,201907,609,689,129426
7,201908,683,552,131429
8,201909,553,576,133185
9,201910,583,624,135475


In [ ]:
%%sql
COPY competencia_03 TO '{{dataset_path}}competencia_03.csv' (FORMAT CSV, HEADER)